In [31]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/exploratory/species_peaks")

library(dplyr)
library(data.table)

myo_meta <- read.csv("/Users/rebecca/sudmant/analyses/myotis/data/myotis_meta.csv")
ortho_table <- fread("/Users/rebecca/sudmant/analyses/myotis/data/ortholog_mapping.csv", data.table = FALSE)
yum_col <- grep("^yum", colnames(ortho_table))
ortho_table[,yum_col] <- gsub("SCAF", "SUPER", ortho_table[,yum_col]) 
colnames(ortho_table)[1:3] <- c("ENSEMBL", "SYMBOL", "Myotis_Alias")


Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [ ]:
## Get peak status of species' orthologous genes:

for (i in seq_along(myo_meta$Abbr)) {
  
  spec_id <- tolower(gsub("Myotis-", "", myo_meta$Field_Name[i]))
  file_path <- list.files(path = "results/data", 
                          pattern = paste0(myo_meta$Abbr[i], "_genes_5000"),
                          full.names = TRUE)
  peaks <- fread(file_path, data.table = FALSE)
  peaks <- peaks[peaks$V3 == "gene",]
  colnames(peaks)[9] <- "Gene"
  peaks$Gene <- sapply(strsplit(peaks$Gene, ";", fixed = TRUE), "[", 1)                                        
  peaks$Gene <- gsub(".*=", paste0(spec_id, "_"), peaks$Gene) 
  peaks <- peaks %>%
    dplyr::group_by(Gene) %>%
    dplyr::reframe(
      Peak_ID = paste(unique(V13), collapse = ", ")
    )
    
  ## Intersect species 1 features with orthologous genes
  spec_col <- grep(paste0(spec_id, "_ID"), colnames(ortho_table))
  spec_ortho_table <- ortho_table[,c(1:3, spec_col),]
  colnames(spec_ortho_table)[4] <- "Gene"
  spec_ortho_table <- spec_ortho_table[spec_ortho_table$Gene %in% peaks$Gene,]
  peak_orthos <- merge(peaks, spec_ortho_table, by = "Gene", all.x = TRUE, sort = FALSE)
  
  fwrite(peak_orthos, file = gsub(".csv", "_orthologs.csv", file_path))
  
}

In [ ]:
## Get list of orthologous genes present in all species, irrespective of peak status

## Get overlap of genes that have peaks:

In [ ]:
## Comparison of peaks in orthologous genes between 2 species:

for (i in seq_along(myo_meta$Abbr)) {
  
  peaks1 <- fread(list.files(path = "results/data", 
                             pattern = paste0(myo_meta$Abbr[i], "_genes_5000.*orthologs"),
                             full.names = TRUE), data.table = FALSE)
    
  peak_cor_list <- lapply(seq_along(myo_meta$Abbr)[-i], function(j) {
    
    peaks2 <- fread(list.files(path = "results/data", 
                               pattern = paste0(myo_meta$Abbr[i], "_genes_5000.*orthologs"),
                               full.names = TRUE), data.table = FALSE)
    
    ## Align features between species
    peaks1 <- peaks1[peaks1$Myotis_Alias %in% peaks2$Myotis_Alias,]
    peaks2 <- peaks2[peaks2$Myotis_Alias %in% peaks1$Myotis_Alias,]
    peaks1 <- peaks1[match(peaks2$Myotis_Alias, peaks1$Myotis_Alias),]
    peaks1$Peak <- TRUE
    peaks1$Peak[peaks1$Peak_ID == "."] <- FALSE
    peaks2$Peak <- TRUE
    peaks2$Peak[peaks2$Peak_ID == "."] <- FALSE
    
    return(cor(peaks1$Peak, peaks2$Peak))
    
  })
  
}